**Step 1. Confirm your desired tuner parameter specs (below)** are correct.

**Step 2. Run `build_docker` script** in base directory.

- If you get an error "An image does not exist locally with the tag," run this command in the base directory:
- `gcloud auth configure-docker us-central1-docker.pkg.dev`
- (Replace the region `us-central1` if necessary)

**Step 3. Execute this notebook**, replacing BUCKET_NAME and making sure the variables in the 2nd cell are correct.

In [6]:
# adapted from: https://io.google/2022/program/934cbc5f-42bb-4f6c-99fe-4972995eb381/
from google.cloud import aiplatform
from google.cloud.aiplatform import hyperparameter_tuning as hpt

In [7]:
STAGING_BUCKET_NAME = "iam-model-staging"  # An already existing bucket in Cloud Storage
PROJECT_ID="handwriting-keras-tuner"  # An already existing Google Cloud project
REGION="us-central1"  # The desired region
REPO="word-models"  # Name of the Artifact Registry repo containing the Docker image
IMAGE="iam-tuner"  # Name of the Docker image
TAG="hypertune"  # Tag used when creating the Docker image

In [7]:
worker_pool_specs = [{
    'machine_spec': {
        None
        # 'machine_type': 'n1-standard-4',
        # 'accelerator_type': 'NVIDIA_TESLA_V100',
        # 'accelerator_count': 1
    },
    'replica_count': 0,  # 1,
    'container_spec': {
        'image_url': f'{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPO}/{IMAGE}:{TAG}'
    }
}]

parameter_spec = {
    'batch_size': hpt.DiscreteParameterSpec(values=[16, 32, 64, 128], scale=None),
    'kernel_size': hpt.DiscreteParameterSpec(values=[2, 3, 4], scale=None),
    'activation': hpt.CategoricalParameterSpec(values=['relu', 'sigmoid', 'tanh']),
    'dropout': hpt.DoubleParameterSpec(min=0.1, max=0.5, scale='linear'),
    'num_units_dense1': hpt.DiscreteParameterSpec(values=[64, 128, 256], scale=None),
    'num_units_ltsm1': hpt.DiscreteParameterSpec(values=[128, 256, 512, 768, 1024], scale=None),
    'num_units_ltsm2': hpt.DiscreteParameterSpec(values=[128, 256, 512, 768, 1024], scale=None),
    'learning_rate': hpt.DoubleParameterSpec(min=0.001, max=1, scale='log'),
    
}

metric_spec = {'val_loss': 'minimize'}

In [ ]:
custom_job = aiplatform.CustomJob(display_name='iam_tuner',
                                 worker_pool_specs=worker_pool_specs,
                                 staging_bucket=f'gs://{STAGING_BUCKET_NAME}')

hp_job = aiplatform.HyperparameterTuningJob(
    display_name='iam_tuner',
    custom_job=custom_job,
    metric_spec=metric_spec,
    parameter_spec=parameter_spec,
    # max_trial_count=2,  # 16,
    parallel_trial_count=2,
    search_algorithm=None
)

hp_job.run()